# 프로젝트: 퍼스널 컬러 진단 - CNN

## 데이터
1. 직접 수집한 쿨톤 100장, 웜톤 100장  

## 목표
1. ColorAutoEncoder을 이용한 쿨톤과 웜톤 분류 (제거됨)
2. 간단한 CNN 분류

# 2.  간단한 CNN 모델

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

import numpy as np
import cv2
import os
import glob
from PIL import Image

In [2]:
# 파일 불러오는 함수
def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기
    image_paths.extend(glob.glob(directory_path + '/*.jpeg'))  # 디렉토리 내의 모든 jpeg 파일 경로 추가

    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)

    return images

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 모든 이미지 불러오기
directory_path = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/쿨톤_jpg"  # 디렉토리 경로
directory_path2 = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/웜톤_jpg"  # 디렉토리 경로
images = load_images_from_directory(directory_path)
images2 = load_images_from_directory(directory_path2)

length = len(images)
print(length)
print(len(images2))

# 이미지 리스트 합치기
images = np.concatenate((images, images2), axis=0)
print(len(images))

50
50
100


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
def preprocess_image(image):
    # 이미지 크기 조절
    image = tf.image.resize(image, (200, 200))
    # 이미지를 [0, 1] 범위로 정규화
    image = image / 255.0
    return image

images = np.array([preprocess_image(image) for image in images])
print(images.shape)

(100, 200, 200, 3)


- KNN과 다르게 평탄화 작업을 제거하였고 리사이즈 함수가 다르다

In [6]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 처음 50개의 요소를 1로 변경
# 0이 쿨톤 1이 웜톤
labels[length:] = 1

# 0이 쿨톤 1이 웜톤
print(len(images))
print(labels.shape)
print(labels)

100
(100,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(images,
                                                  labels,
                                                  test_size=0.3,
                                                  random_state=1024)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (70, 200, 200, 3)
y_train:  (70,)
x_val:  (30, 200, 200, 3)
y_val:  (30,)


In [8]:
# 데이터를 텐서로 변환
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.int32)

In [9]:

# 모델 정의
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 두 개의 클래스로 분류 예시
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [10]:

# 모델 학습
num_epochs = 10
model.fit(x_train, y_train, epochs=num_epochs, batch_size=32)

print("Training finished!")

Epoch 1/10
3/3 [==============================] - 6s 709ms/step - loss: 1.9162 - accuracy: 0.4143
Epoch 2/10
3/3 [==============================] - 3s 791ms/step - loss: 0.9931 - accuracy: 0.5429
Epoch 3/10
3/3 [==============================] - 3s 617ms/step - loss: 0.8147 - accuracy: 0.5429
Epoch 4/10
3/3 [==============================] - 2s 600ms/step - loss: 0.6841 - accuracy: 0.5429
Epoch 5/10
3/3 [==============================] - 3s 1s/step - loss: 0.6773 - accuracy: 0.7429
Epoch 6/10
3/3 [==============================] - 3s 671ms/step - loss: 0.6611 - accuracy: 0.8143
Epoch 7/10
3/3 [==============================] - 2s 612ms/step - loss: 0.6474 - accuracy: 0.5571
Epoch 8/10
3/3 [==============================] - 3s 817ms/step - loss: 0.6091 - accuracy: 0.7429
Epoch 9/10
3/3 [==============================] - 3s 793ms/step - loss: 0.5582 - accuracy: 0.7714
Epoch 10/10
3/3 [==============================] - 3s 1s/step - loss: 0.5116 - accuracy: 0.9143
Training finished!


In [11]:
pred = model.predict(x_val)
pred_class = np.argmax(pred, axis=1)
y_val = np.array(y_val)

1/1 [==============================] - 0s 411ms/step


In [12]:
print(pred_class)
print(y_val)

[0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0]
[0 0 0 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0]


In [13]:
acc = np.mean(pred_class == y_val)
print('accuracy: %f' % (acc,))

accuracy: 0.633333


ramdom_state를 변화시키면서 train, val 나누는 코드부터 반복 시도함  
1차 시도: 0.56  
2차 시도: 0.53  
3차 시도: 0.33  
4차 시도: 0.40  
5차 시도: 0.40

- TEST

In [18]:
class_name = ["쿨톤", "웜톤"]

In [21]:
# test 불러오기 (낱개 버전)
test_path = "/content/drive/MyDrive/four_seasons/사계절_연예인 이미지 데이터셋/prototype/연예인 사진 10장/01.jpg"
x_test = Image.open(test_path)
x_test = np.array(x_test)
print(x_test.shape)

# 배열의 크기 변환
x_test = x_test.reshape((1, x_test.shape[0], x_test.shape[1], 3))
print("변환된 배열 크기:", x_test.shape)

# 전처리(resize, 정규화)
x_test = preprocess_image(x_test)
print(x_test.shape)

# 만약 앱구현과 연결된다면 저장된 모델을 다시 돌릴 필요없이 저장된 최적의 모델을 사용하도록 수정해야함
# test
pred = model.predict(x_test)
pred_class = np.argmax(pred, axis=1)
print("당신은", class_name[int(pred_class)], "입니다~!")
print("쿨톤일 확률: ", pred[:, 0])
print("웜톤일 확률: ", pred[:, 1])

(600, 600, 3)
변환된 배열 크기: (1, 600, 600, 3)
(1, 200, 200, 3)
1/1 [==============================] - 0s 33ms/step
당신은 쿨톤 입니다~!
쿨톤일 확률:  [0.569514]
웜톤일 확률:  [0.43048605]
